In [1]:
!pip install mediapipe==0.10.20

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 20.7 MB/s eta 0:00:00
  Attempting uninstall: mediapipe
    Found existing installation: mediapipe 0.10.21
    Uninstalling mediapipe-0.10.21:
      Successfully uninstalled mediapipe-0.10.21


In [9]:
import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm
from collections import Counter

def detect_and_summarize(video_path, output_path, summary_interval_sec=5):
    mp_face_mesh = mp.solutions.face_mesh
    mp_pose = mp.solutions.pose
    mp_draw = mp.solutions.drawing_utils

    face_mesh = mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.6,
        min_tracking_confidence=0.6
    )

    pose = mp_pose.Pose(
        static_image_mode=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    cap = cv2.VideoCapture(video_path)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    out = cv2.VideoWriter(
        output_path,
        cv2.VideoWriter.fourcc(*'mp4v'),
        fps,
        (width, height)
    )

    frames_per_summary = fps * summary_interval_sec

    emotion_buffer = []
    movement_buffer = []
    summary_text = "Resumo: aguardando..."

    prev_hip_y = None

    for frame_idx in tqdm(range(total_frames)):
        ret, frame = cap.read()
        if not ret:
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # -------------------------
        # EMOÇÃO (FACE MESH)
        # -------------------------
        emotion = "Neutro"
        face_results = face_mesh.process(rgb)

        if face_results.multi_face_landmarks:
            lm = face_results.multi_face_landmarks[0].landmark
            mouth_open = abs(lm[13].y - lm[14].y)
            eye_open = abs(lm[159].y - lm[145].y)

            if mouth_open > 0.035 and eye_open > 0.02:
                emotion = "Surpreso"
            elif mouth_open > 0.025:
                emotion = "Feliz"

            mp_draw.draw_landmarks(
                frame,
                face_results.multi_face_landmarks[0],
                mp_face_mesh.FACEMESH_TESSELATION
            )

        # -------------------------
        # MOVIMENTO (POSE)
        # -------------------------
        movement = "Parado"
        pose_results = pose.process(rgb)

        if pose_results.pose_landmarks:
            lm = pose_results.pose_landmarks.landmark
            left_wrist = lm[mp_pose.PoseLandmark.LEFT_WRIST]
            right_wrist = lm[mp_pose.PoseLandmark.RIGHT_WRIST]
            nose = lm[mp_pose.PoseLandmark.NOSE]
            hip = lm[mp_pose.PoseLandmark.LEFT_HIP]

            if left_wrist.y < nose.y or right_wrist.y < nose.y:
                movement = "Braços levantados"

            if prev_hip_y is not None:
                if abs(prev_hip_y - hip.y) > 0.01:
                    movement = "Andando"

            prev_hip_y = hip.y

            mp_draw.draw_landmarks(
                frame,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS
            )

        # -------------------------
        # BUFFER PARA RESUMO
        # -------------------------
        emotion_buffer.append(emotion)
        movement_buffer.append(movement)

        # -------------------------
        # GERAR RESUMO A CADA 5s
        # -------------------------
        if frame_idx % frames_per_summary == 0 and frame_idx > 0:
            dominant_emotion = Counter(emotion_buffer).most_common(1)[0][0]
            dominant_movement = Counter(movement_buffer).most_common(1)[0][0]

            start_sec = (frame_idx - frames_per_summary) // fps
            end_sec = frame_idx // fps

            summary_text = (
                f"Resumo (Entre {start_sec}s e {end_sec}s): "
                f"Emocao: {dominant_emotion} | Movimento: {dominant_movement}"
            )

            print(summary_text)

            emotion_buffer.clear()
            movement_buffer.clear()

        # -------------------------
        # TEXTO NO VÍDEO
        # -------------------------
        cv2.rectangle(frame, (10, height - 80), (width - 10, height - 20), (0, 0, 0), -1)

        cv2.putText(
            frame,
            summary_text,
            (20, height - 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 255),
            2
        )

        out.write(frame)

    cap.release()
    out.release()
    face_mesh.close()
    pose.close()

In [10]:
detect_and_summarize('/content/video.mp4', '/content/video_com_emocoes_e_movimentos.mp4', 1)

  1%|          | 33/3326 [00:01<03:01, 18.16it/s]

Resumo (Entre 0s e 1s): Emocao: Neutro | Movimento: Parado


  2%|▏         | 63/3326 [00:03<02:52, 18.87it/s]

Resumo (Entre 1s e 2s): Emocao: Neutro | Movimento: Parado


  3%|▎         | 93/3326 [00:05<02:51, 18.84it/s]

...

 98%|█████████▊| 3275/3326 [02:26<00:01, 25.92it/s]

Resumo (Entre 108s e 109s): Emocao: Neutro | Movimento: Parado


 99%|█████████▉| 3302/3326 [02:27<00:00, 24.07it/s]

Resumo (Entre 109s e 110s): Emocao: Neutro | Movimento: Parado


100%|██████████| 3326/3326 [02:29<00:00, 22.29it/s]
